<a href="https://colab.research.google.com/github/romapavelko01/LAB_4/blob/master/test_task_MACPAW_Pavelko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
# from statsmodels.stats.diagnostic import het_breuschpagan
# from statsmodels.stats.diagnostic import het_white
# from statsmodels.formula.api import ols

train_df = pd.read_csv('https://raw.githubusercontent.com/MacPaw/msi2021-data-science/main/data/train.csv')


test_df = pd.read_csv('https://raw.githubusercontent.com/MacPaw/msi2021-data-science/main/data/test.csv')
test_prices = pd.read_csv('https://raw.githubusercontent.com/MacPaw/msi2021-data-science/main/data/sample_prediction.csv')

test_df['SalePrice'] = test_prices['SalePrice']

all_df = pd.concat([train_df, test_df])
all_df.drop('Id', axis=1, inplace=True)

# creating a concatenated dataframe, encoding all categorical columns,
# splitting the dataframe into train and test data
# building a model on train data and then 
def convert_all_categorical(inp_df):
    """
    Returns a new dataframe with encoded categorical columns.
    To avoid multicollinearity, one needs to drop one variable -
    for a column with n different categories, there will be created n - 1 columns.
    """
    categorical_columns = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape',
                       'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
                       'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
                       'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle',
                       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
                       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
                       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                       'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
                       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
                       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive',
                       'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
    return pd.get_dummies(all_df, columns=categorical_columns, prefix=categorical_columns, drop_first=True)


def clean_dataset(df):
    """
    Needed for preventing ValueError:
    Input contains NaN, infinity or a value too large for dtype('float64')
    """
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)


# Was trying to conduct tests for heteroskedasticity,
# but the White test was making the session run out of memory, while the 
# Breusch-Pagan test, being more time-efficient, raised some error I could not 
# resolve.


def relevant_variables(inp_df):
    """
    Function for choosing the important variables for a linear regression model
    based on the p-value (which should be < 0.05, otherwise - neglected).
    """
    Y_1 = inp_df[['SalePrice']]
    Y = np.array(Y_1)
    X = inp_df.copy()
    X.drop('SalePrice', axis=1, inplace=True)
    X_1 = np.array(X)
    X2 = sm.add_constant(X_1)
    est = sm.OLS(Y, X2).fit()
    indices_to_keep = list()
    p_values_list = est.pvalues
    for i in range(1, len(p_values_list)):
        if p_values_list[i] < 0.05:
            indices_to_keep.append(i - 1) 
            # since 0-th element is the intercept
            # and we want to keep the indices of the columns that are relevant
            # by p-value
    X_columns = X.columns
    cols_to_keep = [X_columns[idx] for idx in indices_to_keep]
    return X[cols_to_keep]


converted_all_df = convert_all_categorical(all_df).reset_index()
final_all_df = clean_dataset(converted_all_df)
train, test = train_test_split(final_all_df, test_size=0.5)


train, test = relevant_variables(train), relevant_variables(test)












      LotArea  YearBuilt  ...  PoolQC_Gd  SaleCondition_AdjLand
1669  13514.0     2008.0  ...        0.0                    0.0
1      9600.0     1976.0  ...        0.0                    0.0
547    7244.0     1970.0  ...        0.0                    0.0
744    5395.0     1993.0  ...        0.0                    0.0
2400   5175.0     1958.0  ...        0.0                    0.0
...       ...        ...  ...        ...                    ...
585   11443.0     2005.0  ...        0.0                    0.0
63    10300.0     1921.0  ...        0.0                    0.0
779   10530.0     1977.0  ...        0.0                    0.0
1046  16056.0     2005.0  ...        0.0                    0.0
271   39104.0     1954.0  ...        0.0                    0.0

[1133 rows x 33 columns]


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
